In [1]:
import pandas as pd
import numpy as np
import requests
import undetected_chromedriver as uc
import matplotlib.pyplot as plt
import time
import psutil
import gc
import re

from datetime import datetime, timedelta
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from wordcloud import WordCloud 
 


# 셀레니움 버전

In [ ]:
# uc 안쓰고 그냥 셀레니움 웹드라이버 사용시
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # 브라우저를 표시하지 않음
# driver = webdriver.Chrome(options=options)

## new start

In [34]:
# 리소스 정리

# 자바스크립트를 사용하여 ul 요소의 스타일을 변경하여 표시
category1_ul_xpath = '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[1]/ul'
category2_ul_xpath = '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[2]/ul'
startmonth_ul_xpath = '//*[@id="content"]/div[2]/div/div[1]/div/div/div[2]/div[2]/span[1]/div[2]/ul'
startday_ul_xpath = '//*[@id="content"]/div[2]/div/div[1]/div/div/div[2]/div[2]/span[1]/div[3]/ul'
endmonth_ul_xpath = '//*[@id="content"]/div[2]/div/div[1]/div/div/div[2]/div[2]/span[3]/div[2]/ul'
endday_ul_xpath = '//*[@id="content"]/div[2]/div/div[1]/div/div/div[2]/div[2]/span[3]/div[3]/ul'
top100_ul_xpath = '//*[@id="content"]/div[2]/div/div[2]/div[2]/div/div/div[1]/ul'
next_btn_xpath = '//*[@id="content"]/div[2]/div/div[2]/div[2]/div/div/div[2]/div/a[2]'
check_btn_xpath = '//*[@id="content"]/div[2]/div/div[1]/div/a'
gender_selectID = ['19_gender_0','19_gender_1','19_gender_2']
age_selectID = ['20_age_0','20_age_1','20_age_2','20_age_3','20_age_4','20_age_5','20_age_6']
category_list = ['축산물','수산물','농산물','반찬','김치','음료','과자/베이커리','유가공품','냉동/간편조리식품','건강식품','다이어트식품','통조림/캔','제과/제빵재료','조미료','식용유/오일','소스/드레싱','가루/분말류','잼/시럽','라면/면류','장류','밀키트','주류']
# category_list = ['축산물','수산물','농산물','반찬','김치']
# category_select = '축산물'

# 날짜 범위 설정
start_date = datetime(2024, 5, 1)
# end_date = datetime.now() - timedelta(days=1) # 크롤링 실행날짜 -1 day
end_date = datetime(2024, 5, 31)
delta = timedelta(days=1)
current_date = end_date

In [35]:
# Selenium WebDriver 설정
options = uc.ChromeOptions()
options.add_argument('--headless')  # 브라우저를 표시하지 않음
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = uc.Chrome(options=options, use_subprocess=False)
wait = WebDriverWait(driver, 10)
# 네이버 데이터랩 쇼핑인사이트 URL 접속
url = "https://datalab.naver.com/shoppingInsight/sCategory.naver"
driver.get(url)


In [20]:
def initialize_driver():
    options = uc.ChromeOptions()
    # options.add_argument('--headless')  # 브라우저를 표시하지 않음
    # options.add_argument('--no-sandbox')
    # options.add_argument('--disable-dev-shm-usage')
    # driver = uc.Chrome(options=options, use_subprocess=False)
    driver = uc.Chrome(use_subprocess=False)
    driver.get('https://datalab.naver.com/shoppingInsight/sCategory.naver')  # 페이지 URL로 이동
    return driver

def open_ul(xpath):
    ul_element = driver.find_element(By.XPATH, xpath)
    driver.execute_script("arguments[0].style.display = 'block';", ul_element)

def close_driver(driver):
    driver.quit()
    for proc in psutil.process_iter():
        if proc.name() == "chromedriver" or proc.name() == "chrome":
            proc.kill()
    gc.collect()

# 모든 체크박스를 false로 설정하는 함수
def uncheck_all_checkboxes(checkbox_ids):
    for checkbox_id in checkbox_ids:
        checkbox = driver.find_element(By.ID, checkbox_id)
        driver.execute_script("arguments[0].checked = false;", checkbox)


In [36]:
# 전부 열기
open_ul_list = [category1_ul_xpath, category2_ul_xpath, startmonth_ul_xpath, startday_ul_xpath, endmonth_ul_xpath, endday_ul_xpath]
[open_ul(i) for i in open_ul_list]

# 1분류 식품 선택
category1_elements_driver = driver.find_element(By.XPATH, category1_ul_xpath)
category1_elements = category1_elements_driver.find_elements(By.TAG_NAME, 'li')
a_element = [li.find_element(By.TAG_NAME, 'a') for li in category1_elements if li.find_element(By.TAG_NAME, 'a').text == '식품'][0]
driver.execute_script("arguments[0].click();", a_element)
# 1분류 첫번째 카테고리의 2분류 li 갯수가 4개라 바로 크롤링할 경우 식품의 4번째까지만 진행하는 오류로 대기시간 설정
time.sleep(1)

# 날짜 선택은 start와 end를 묶어서 역순으로 진행 - 과거부터 진행하면 end_li 선택시 start_li가 사라짐
# start month, day 선택
# 2분류 하나씩 해야 오류가 안생길듯
# for a_element in [li.find_element(By.TAG_NAME, 'a') for li in category2_elements]:
category2_elements_driver = driver.find_element(By.XPATH, category2_ul_xpath)
category2_elements = category2_elements_driver.find_elements(By.TAG_NAME, 'li')

results = []

for category_select in category_list:
    desired_element = [li.find_element(By.TAG_NAME, 'a') for li in category2_elements if li.find_element(By.TAG_NAME, 'a').text == category_select][0]
    driver.execute_script("arguments[0].click();", desired_element)
    print(category_select) # 진행상황 확인
    
    for gender in gender_selectID: # gender 선택
        uncheck_all_checkboxes(gender_selectID) # 모든 체크박스 해제 후 성별 체크박스 루프 진행
        gender_checkbox = driver.find_element(By.ID, gender)
        driver.execute_script("arguments[0].click();", gender_checkbox)

        
        for age in age_selectID: # age 선택, 50대, 60대 그룹핑
            if age == '20_age_6':
                continue
            uncheck_all_checkboxes(age_selectID)
            if age not in ['20_age_5', '20_age_6']:
                age_checkbox = driver.find_element(By.ID, age)
                driver.execute_script("arguments[0].click();", age_checkbox)
            else:
                age_checkbox_5 = driver.find_element(By.ID, '20_age_5')
                age_checkbox_6 = driver.find_element(By.ID, '20_age_6')
                driver.execute_script("arguments[0].click();", age_checkbox_5)
                driver.execute_script("arguments[0].click();", age_checkbox_6)

            # 날짜 선택 및 데이터 수집
            current_date = end_date
            while current_date >= start_date:
                month_str = current_date.strftime('%m')
                day_str = current_date.strftime('%d')

                # 시작 월과 일 선택
                startM_elements_driver = driver.find_element(By.XPATH, startmonth_ul_xpath)
                startM_elements = startM_elements_driver.find_elements(By.TAG_NAME, 'li')
                startM_element = [li.find_element(By.TAG_NAME, 'a') for li in startM_elements if li.find_element(By.TAG_NAME, 'a').text == month_str][0]
                driver.execute_script("arguments[0].click();", startM_element)

                startD_elements_driver = driver.find_element(By.XPATH, startday_ul_xpath)
                startD_elements = startD_elements_driver.find_elements(By.TAG_NAME, 'li')
                startD_element = [li.find_element(By.TAG_NAME, 'a') for li in startD_elements if li.find_element(By.TAG_NAME, 'a').text == day_str][0]
                driver.execute_script("arguments[0].click();", startD_element)

                # 종료 월과 일 선택
                endM_elements_driver = driver.find_element(By.XPATH, endmonth_ul_xpath)
                endM_elements = endM_elements_driver.find_elements(By.TAG_NAME, 'li')
                endM_element = [li.find_element(By.TAG_NAME, 'a') for li in endM_elements if li.find_element(By.TAG_NAME, 'a').text == month_str][0]
                driver.execute_script("arguments[0].click();", endM_element)

                endD_elements_driver = driver.find_element(By.XPATH, endday_ul_xpath)
                endD_elements = endD_elements_driver.find_elements(By.TAG_NAME, 'li')
                endD_element = [li.find_element(By.TAG_NAME, 'a') for li in endD_elements if li.find_element(By.TAG_NAME, 'a').text == day_str][0]
                driver.execute_script("arguments[0].click();", endD_element)

                # 조회하기 클릭
                check_element = driver.find_element(By.XPATH, check_btn_xpath)
                driver.execute_script("arguments[0].click();", check_element)
                time.sleep(0.5)
                
                # TOP100 ul 선택, 리스트 컴프리헨션을 사용하여 중간 리스트 생성
                top100_elements_driver = driver.find_element(By.XPATH, top100_ul_xpath)
                date_str = current_date.strftime('%Y-%m-%d')

                # 처음 20개의 요소 가져오기
                top100_elements = top100_elements_driver.find_elements(By.TAG_NAME, 'li')
                intermediate_results = [{'date': date_str, 'category': category_select,'gender': gender,'age': age, 'rank': rank + 1, 'keyword': re.sub(r'\d+', '', li.text).strip()} for rank, li in enumerate(top100_elements)]

                # top100을 뽑기위해 next 버튼 4번 클릭하여 80개의 요소 추가로 가져오기
                for i in range(4):
                    next_btn_element = driver.find_element(By.XPATH, next_btn_xpath)
                    driver.execute_script("arguments[0].click();", next_btn_element)
                    time.sleep(0.5)
                    top100_elements_driver = driver.find_element(By.XPATH, top100_ul_xpath)
                    top100_elements = top100_elements_driver.find_elements(By.TAG_NAME, 'li')
                    intermediate_results += [{'date': date_str, 'category': category_select,'gender': gender,'age': age, 'rank': len(intermediate_results) + rank + 1, 'keyword': re.sub(r'\d+', '', li.text).strip()} for rank, li in enumerate(top100_elements)]
                    
                # 기존 results 리스트에 중간 리스트의 항목들을 확장
                results.extend(intermediate_results)

                # 날짜를 하루 뒤로 이동
                current_date -= delta

            # 종료 월과 일 선택 (for 루프의 다음 반복을 위해 초기화)
            month_str = end_date.strftime('%m')
            day_str = end_date.strftime('%d')

            endM_elements_driver = driver.find_element(By.XPATH, endmonth_ul_xpath)
            endM_elements = endM_elements_driver.find_elements(By.TAG_NAME, 'li')
            endM_element = [li.find_element(By.TAG_NAME, 'a') for li in endM_elements if li.find_element(By.TAG_NAME, 'a').text == month_str][0]
            driver.execute_script("arguments[0].click();", endM_element)

            endD_elements_driver = driver.find_element(By.XPATH, endday_ul_xpath)
            endD_elements = endD_elements_driver.find_elements(By.TAG_NAME, 'li')
            endD_element = [li.find_element(By.TAG_NAME, 'a') for li in endD_elements if li.find_element(By.TAG_NAME, 'a').text == day_str][0]
            driver.execute_script("arguments[0].click();", endD_element)

close_driver(driver)

In [37]:
df = pd.DataFrame(results)
df

,date,category,gender,age,rank,keyword
0,2024-05-31,축산물,19_gender_0,20_age_0,1,닭가슴살
1,2024-05-31,축산물,19_gender_0,20_age_0,2,살치살
2,2024-05-31,축산물,19_gender_0,20_age_0,3,우대갈비
3,2024-05-31,축산물,19_gender_0,20_age_0,4,꽃갈비살
4,2024-05-31,축산물,19_gender_0,20_age_0,5,삼겹살
...,...,...,...,...,...,...
1172211,2024-05-01,주류,19_gender_2,20_age_5,96,담은
1172212,2024-05-01,주류,19_gender_2,20_age_5,97,맥캘란년
1172213,2024-05-01,주류,19_gender_2,20_age_5,98,맵시막걸리
1172214,2024-05-01,주류,19_gender_2,20_age_5,99,술선물


In [38]:
df.to_csv('results.csv', index=False)

In [31]:
close_driver(driver)

In [62]:
df_word = df
df_word = df_word.groupby('keyword', as_index = False).count()
df_word

,keyword,date,category,rank
0,bbq닭다리,2,2,2
1,bbq자메이카통다리,15,15,15
2,bbq치즈시즈닝,2,2,2
3,bhc닭가슴살,2,2,2
4,cj부드러운닭가슴살소세지,2,2,2
...,...,...,...,...
687,훈제닭다리,2,2,2
688,훈제란,17,17,17
689,훈제메추리알,2,2,2
690,훈제오리,200,200,200


In [ ]:
# 각 li 요소의 텍스트 출력
for li in li_elements:
    a_tag = li.find_element(By.TAG_NAME, 'a')
    print(a_tag.text)

# 원하는 동작 수행 (예: 특정 li 요소 클릭)
for li in li_elements:
    a_tag = li.find_element(By.TAG_NAME, 'a')
    a_tag.click()

## 아래 코드는 test용입니다. 구분선

In [ ]:
# 주어진 XPath를 사용하여 span 요소를 찾음
element_xpath = '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[2]/span'
span_element = driver.find_element(By.XPATH, element_xpath)

# 원하는 동작 수행 (예: 텍스트 변경)
new_text = '남성의류'
driver.execute_script("arguments[0].textContent = arguments[1];", span_element, new_text)

In [ ]:
# Selenium WebDriver 설정
options = uc.ChromeOptions()
# options.add_argument('--headless')  # 브라우저를 표시하지 않음
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = uc.Chrome(options=options, use_subprocess=False)
wait = WebDriverWait(driver, 10)

# 네이버 데이터랩 쇼핑인사이트 URL 접속
url = "https://datalab.naver.com/shoppingInsight/sCategory.naver"
driver.get(url)

# 식품 카테고리 설정
select_category1 = '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[1]/span'
select_category1_eat = '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[1]/ul/li[7]/a'
wait.until(EC.element_to_be_clickable((By.XPATH, select_category1))).click()
wait.until(EC.element_to_be_clickable((By.XPATH, select_category1_eat))).click()

# 2분류 카테고리
select_category2 = '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[2]/span'
select_category2_box = '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[2]/ul'

results = []
buttons = driver.find_elements(By.XPATH, "//span[@data-id]")

for i in range(1, 3):  # li[1]부터 li[22]까지 순환
    wait.until(EC.element_to_be_clickable((By.XPATH, select_category2))).click()
    category_xpath = f'//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[2]/ul/li[{i}]/a'
    category_element = wait.until(EC.element_to_be_clickable((By.XPATH, category_xpath)))
    category_text = category_element.text
    driver.execute_script("arguments[0].click();", category_element)

    # 날짜 범위 설정 (2024-01-01 ~ 2024-06-06)
    start_date = datetime(2024, 6, 5)
    end_date = datetime(2024, 6, 6)
    current_date = start_date

    while current_date <= end_date:
        month = current_date.strftime('%m')
        day = current_date.strftime('%d')

        # 날짜 설정 함수 호출
        set_date(month, day)

        # 조회 버튼 클릭
        select_submit_btn = '//*[@id="content"]/div[2]/div/div[1]/div/a'
        wait.until(EC.element_to_be_clickable((By.XPATH, select_submit_btn))).click()

        time.sleep(1)  # 페이지 로딩을 기다리기 위해 잠시 대기

        # 랭킹 리스트 추출
        rank_list = '//*[@id="content"]/div[2]/div/div[2]/div[2]/div/div/div[1]/ul'
        items = driver.find_elements(By.XPATH, f'{rank_list}/li')

        for item in items:
            rank = item.find_element(By.CSS_SELECTOR, '.rank_top1000_num').text
            keyword = item.text.replace(rank, '').strip()
            results.append({'date': current_date.strftime('%Y-%m-%d'), 'rank': rank, 'keyword': keyword, 'category': category_text})

        # 다음 날짜로 이동
        current_date += timedelta(days=1)

    # 다음 카테고리 선택
    wait.until(EC.element_to_be_clickable((By.XPATH, select_category2))).click()

# 드라이버 종료
driver.quit()

# DataFrame으로 변환 및 CSV 파일로 저장
df = pd.DataFrame(results)
df

In [ ]:
start_date = datetime(2024, 5, 4)
end_date = datetime(2024, 6, 6)
current_date = start_date
month = current_date.strftime('%m')
day = current_date.strftime('%d')

In [ ]:
month, day

## 2분류 까지

# BeautifulSoup 버전
- 동적 제어가 필요하기 때문에 해당 뷰슾으로는 식품 인기검색어를 뽑을 수 없음

In [ ]:
# # User-Agent 설정
# headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'}

# # 네이버 데이터랩 쇼핑인사이트 URL
# url = "https://datalab.naver.com/shoppingInsight/sCategory.naver"

# # HTML 요청 및 파싱
# html = requests.get(url=url, headers=headers).text
# soup = BeautifulSoup(html, 'html.parser')

# # 아이템 선택
# items = soup.select('ul.rank_top1000_list li')

# # 결과 저장 리스트
# results = []

# # 아이템 순회하며 데이터 추출
# for item in items:
#     rank = item.select_one('.rank_top1000_num').text
#     keyword = item.get_text(strip=True).replace(rank, '')
#     results.append({'rank': rank, 'keyword': keyword})

# print(soup)